In [60]:
#  Auteur: Dina Benkirane
#          Zheng Long Yang
#Note: can't encrypt strings nor decrypt, since i couldnt create OAEP in time.

import random
import math
import hashlib


class RSA:
    def __init__(self, message_clair):
        self.message_clair= int(message_clair,10)
        n,e,p,q,d,dp,dq,qInv=self.genKeys()
        print("Test Function")
        #Donne des chiffres negatif
        ciphertext=self.squareAndMultiply(message_clair,e,n)
        print("Message encrypter avec OAEP: ",ciphertext)
        decryptText2=self.squareAndMultiply(ciphertext,d,n)
        
        #chinese remainder not working
        #decryptText=self.Decryption(p,q,dp,dq,qInv,ciphertext)
        print("Message decrypter avec la fonction CRT: ",decryptText2)
        
        
    #pour efectuer l'algorithme d'exponentiation rapide. a^e%n
    def squareAndMultiply(self, base, exposant, number):
        # Initialisation variable et cas de base
        resultat = 1
        if (exposant == 0):
            return resultat

        # change exposant --> binaire
        binaryexp = []
        while (exposant != 0):
            #print("exposant ", exposant)
            binaryexp.append(exposant % 2)
            exposant = exposant // 2
        #print("Current Binary length: ",binaryexp.__len__())

        # Algo
        if(binaryexp[0]==1):
            resultat=base
        if (binaryexp.__len__()==1):
            return resultat
        for x in range(1,binaryexp.__len__()):
            base = pow(base, 2) % number
            #print("base",base)
            if (binaryexp[x] == 1):
                resultat = base * resultat % number
                #print("resultat if",resultat)
        return resultat
    
    #calcule le pgcd() entre deux nombre a l'aide de l'algorithme d'euclide etendu.
    def EEA(self,a,b):
        #cas de base
        if (b==0):
            d=a
            x=1
            y=0
            return (d,x,y)
        #initialisation de variable
        x2=1
        x1=0
        y2=0
        y1=1
        #trouve un x et y ou ax+by=d
        while(b>0):
            q= a//b
            r=a-q*b
            x=x2-q*x1
            y=y2-q*y1

            a=b
            b=r
            x2=x1
            x1=x
            y2=y1
            y1=y
            
        d=a
        x=x2
        y=y2
        return (d,x,y)
    
    #permet de calculer l'inverse modulaire
    def modinv(self,e,phi):
        d,x,y=self.EEA(e,phi)
        #if (d!=1):
        #    raise ValueError ("ModInv doesn't exist")
        return x%phi
    
    #vous permettant de verifer si un nombre est premier.
    def millerRabin(self, number, iteration):
        #    Cas d'exception
        if (number == 2 or number==1 or number==3):
            return True
        if (number%2==0):
            return False

        r, exposantPrime = 0, number - 1
        while exposantPrime % 2 == 0:
            r += 1
            exposantPrime //= 2
        for _ in range(iteration):
            randInt = random.randrange(2, number - 1)
            modExp = self.squareAndMultiply(randInt,exposantPrime,number)
            if modExp == 1 or modExp == number - 1:
                continue
            for _ in range(r - 1):
                modExp = self.squareAndMultiply(modExp,2,number)
                if modExp == number - 1:
                    break
            else:
                return False
        return True
        
    #Create a prime number
    def getPrimeNumber(self,length):
        prime=random.getrandbits(length)
        #A partir du nombre genere diminuer de 1 jusqua temps quon trouve un nombre
        #premier
        while(self.millerRabin(prime,40)==False):
            prime-=1
            #print("prime",prime)
        return prime

    #etablie tous les parametres initiaux requis.
    def genKeys(self):
        p=-1
        #32 car 32^2 est 1024, la valeur minimal que n doit avoir
        q=self.getPrimeNumber(513)
        nMod=p*q
        #genere un p qui va donner 
        while(len(bin(nMod))<1024 and self.EEA(p-1,q-1)==2):
            p=self.getPrimeNumber(513)
            nMod=p*q
        print(nMod)
            #print("N= ",len(bin(nMod)))
            
        #print("p: ",p," q: ",q)
        
        #Calcul de Phi de n
        phiN =(p-1)*(q-1)
        #print("phiN", phiN)
        
        #Choisir l'exposant e petit
        exponent=random.randrange(3,102)
        
        while(self.EEA(exponent,phiN)[1]!=1):
            exponent= random.randint
            #accélère la procédure
            if (divmod(exponent,2)==0):
                exponent-1
                
        #calculer Clé privé d
        d=self.modinv(exponent,phiN)
        print("d:", d)
        
        #chinese reminder theorem keys
        dP=self.modinv(d,p-1)
        dQ=self.modinv(d,q-2)
        qInv= self.modinv(q,p)
        
        #retourne les clefs public et priver. pour tenter de faire fonctionner
        return nMod,exponent,d,dP,dQ,qInv,p,q
        
    #Application du chinese reminder theorem    
    def Decryption(self,p,q,dp,dq,qinv,Enc_m):
        m1=self.squareAndMultiply(Enc_m,dp,p)
        m2=self.squareAndMultiply(Enc_m,dq,q)
        h=(qinv*(m1-m2))%p
        message_decrypter= m2+h*q
        return message_decrypter
          
 
    
    #Implementation du I2OSP utiliser dans la fonction masGenF1
    def I2OSP(self, integer, int_length):
        
        if(integer>=pow(256, int_length)):
            raise ValueError("Integer trop grand")
        
        byte_string= ''

        for x in reversed(range(int_length)):
            byte_string.join([chr((integer>>(8 * x) & 0xFF))])
        return byte_string
        
    
    #using sha-256 using the hashlib library to import the function.
    #Generate a mask function, MGF1
    def maskGenF1(self,mgf_seed,mask_len,hash=hashlib.sha256):
        #256 bits is 32 bytes
        hash_len=32
        T=''
        mask=''
        if(mask_len>pow(2,32)*hash_len):
           raise ValueError ("Mask trop grand")
        T=""
        
        for x in range (0,(math.ceil(mask_len/hash_len))):
            C=self.I2OSP(x,4)
            mask+= hash(mgf_seed+C).digest()
            
        return mask[:mask_len]

    #using a sha-256 hash function using the hashlib library   
    #couldnt finish
    def OAEP(self,message_clair):
        hash=hashlib.sha256
        n,e= self.genKeys()
        label = ""
        k_nbitlen = len(bit(n)) 
        m_size = len(message_clair.encode('utf-8'))
        
        if(m_length>(k_nbitlen-2*hashLen-2)):
            raise ValueError ("message trop long")
        #PS, seed, dbMask, maskedDB, seedMask, maskedSeed, and EM
        labelH=hash(label).digest()
        
        
        return message_encrypter

In [61]:
#some test
test=RSA("9187509234123098")
#test2=RSA("")
#test3=RSA(2)
#Test de fonction
#print(test.millerRabin(0,10))
#print(test.millerRabin(133182734227657573431,40))
#print (test.squareAndMultiply(5,596,1234))
#print(test.EEA(2,31))    
#print(test.getPrimeNumber(32))
print(test.genKeys())



-23589539590524789549470431982154974013218592828292515761041027923375299463875947194018811722532309716584950998228183565528465657014168201787773701686515587
d: -47179079181049579098940863964309948026437185656585031522082055846750598927751894388037623445064619433169901996456367131056931314028336403575547403373031171
Test Function


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'